In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
from typing import *
import torch
import torch.optim as optim
import numpy as np
import pandas as pd
from functools import partial
from overrides import overrides

from allennlp.data import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.nn import util as nn_util

In [3]:
class Config(dict):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        for k, v in kwargs.items():
            setattr(self, k, v)
    
    def set(self, key, val):
        self[key] = val
        setattr(self, key, val)
        
config = Config(
    testing=True,
    seed=1,
    batch_size=64,
    lr=3e-4,
    epochs=2,
    hidden_sz=64,
    max_seq_len=100, # necessary to limit memory usage
    max_vocab_size=100000,
)

In [4]:
from allennlp.common.checks import ConfigurationError

In [5]:
USE_GPU = torch.cuda.is_available()

In [6]:
DATA_ROOT = Path("./data") / "jigsaw"

Set random seed manually to replicate results

In [7]:
torch.manual_seed(config.seed)

# Load Data

In [8]:
from allennlp.data.vocabulary import Vocabulary
from allennlp.data.dataset_readers import DatasetReader

### Prepare dataset

In [9]:
label_cols = ["toxic", "severe_toxic", "obscene",
              "threat", "insult", "identity_hate"]

In [10]:
from allennlp.data.fields import TextField, MetadataField, ArrayField

class JigsawDatasetReader(DatasetReader):
    def __init__(self, tokenizer: Callable[[str], List[str]]=lambda x: x.split(),
                 token_indexers: Dict[str, TokenIndexer] = None,
                 max_seq_len: Optional[int]=config.max_seq_len) -> None:
        super().__init__(lazy=False)
        self.tokenizer = tokenizer
        self.token_indexers = token_indexers or {"tokens": SingleIdTokenIndexer()}
        self.max_seq_len = max_seq_len

    @overrides
    def text_to_instance(self, tokens: List[Token], id: str=None,
                         labels: np.ndarray=None) -> Instance:
        sentence_field = TextField(tokens, self.token_indexers)
        fields = {"tokens": sentence_field}
        
        id_field = MetadataField(id)
        fields["id"] = id_field
        
        if labels is None:
            labels = np.zeros(len(label_cols))
        label_field = ArrayField(array=labels)
        fields["label"] = label_field

        return Instance(fields)
    
    @overrides
    def _read(self, file_path: str) -> Iterator[Instance]:
        df = pd.read_csv(file_path)
        if config.testing: df = df.head(1000)
        for i, row in df.iterrows():
            yield self.text_to_instance(
                [Token(x) for x in self.tokenizer(row["comment_text"])],
                row["id"], row[label_cols].values,
            )

### Prepare token handlers

We will use the spacy tokenizer here

In [11]:
from allennlp.data.tokenizers.word_splitter import SpacyWordSplitter
from allennlp.data.token_indexers.elmo_indexer import ELMoCharacterMapper, ELMoTokenCharactersIndexer

# the token indexer is responsible for mapping tokens to integers
token_indexer = ELMoTokenCharactersIndexer()

def tokenizer(x: str):
    return [w.text for w in
            SpacyWordSplitter(language='en_core_web_sm', 
                              pos_tags=False).split_words(x)[:config.max_seq_len]]

In [12]:
reader = JigsawDatasetReader(
    tokenizer=tokenizer,
    token_indexers={"tokens": token_indexer}
)

In [13]:
train_ds, test_ds = (reader.read(DATA_ROOT / fname) for fname in ["train.csv", "test_proced.csv"])
val_ds = None

267it [00:01, 149.56it/s]
251it [00:01, 248.04it/s]


In [14]:
len(train_ds)

267

In [15]:
train_ds[:10]

In [16]:
vars(train_ds[0].fields["tokens"])

{'tokens': [Explanation,
  Why,
  the,
  edits,
  made,
  under,
  my,
  username,
  Hardcore,
  Metallica,
  Fan,
  were,
  reverted,
  ?,
  They,
  were,
  n't,
  vandalisms,
  ,,
  just,
  closure,
  on,
  some,
  GAs,
  after,
  I,
  voted,
  at,
  New,
  York,
  Dolls,
  FAC,
  .,
  And,
  please,
  do,
  n't,
  remove,
  the,
  template,
  from,
  the,
  talk,
  page,
  since,
  I,
  'm,
  retired,
  now.89.205.38.27],
 '_token_indexers': {'tokens': <allennlp.data.token_indexers.elmo_indexer.ELMoTokenCharactersIndexer at 0x7fd8483146a0>},
 '_indexed_tokens': None,
 '_indexer_name_to_indexed_token': None}

### Prepare vocabulary

We don't need to build the vocab: all that is handled by the token indexer

In [17]:
vocab = Vocabulary()

### Prepare iterator

The iterator is responsible for batching the data and preparing it for input into the model. We'll use the BucketIterator that batches text sequences of smilar lengths together.

In [18]:
from allennlp.data.iterators import BucketIterator

In [19]:
iterator = BucketIterator(batch_size=config.batch_size, 
                          sorting_keys=[("tokens", "num_tokens")],
                         )

We need to tell the iterator how to numericalize the text data. We do this by passing the vocabulary to the iterator. This step is easy to forget so be careful! 

In [20]:
iterator.index_with(vocab)

### Read sample

In [21]:
batch = next(iter(iterator(train_ds)))

In [22]:
batch

{'tokens': {'tokens': tensor([[[259,  73, 112,  ..., 261, 261, 261],
           [259, 100,  98,  ..., 261, 261, 261],
           [259, 112, 111,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259, 118, 260,  ..., 261, 261, 261],
           [259, 115, 260,  ..., 261, 261, 261],
           [259,  98, 260,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          [[259,  74, 260,  ..., 261, 261, 261],
           [259,  71,  74,  ..., 261, 261, 261],
           [259,  74,  85,  ..., 261, 261, 261],
           ...,
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0],
           [  0,   0,   0,  ...,   0,   0,   0]],
  
          ...,
  
          [[259,  35

In [23]:
batch["tokens"]["tokens"]

tensor([[[259,  73, 112,  ..., 261, 261, 261],
         [259, 100,  98,  ..., 261, 261, 261],
         [259, 112, 111,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259, 118, 260,  ..., 261, 261, 261],
         [259, 115, 260,  ..., 261, 261, 261],
         [259,  98, 260,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        [[259,  74, 260,  ..., 261, 261, 261],
         [259,  71,  74,  ..., 261, 261, 261],
         [259,  74,  85,  ..., 261, 261, 261],
         ...,
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0],
         [  0,   0,   0,  ...,   0,   0,   0]],

        ...,

        [[259,  35, 260,  ..., 261, 261, 261],
         [259,  81, 109,  ..., 261, 261, 261]

In [24]:
batch["tokens"]["tokens"].shape

torch.Size([64, 44, 50])

# Prepare Model

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim

In [26]:
from allennlp.modules.seq2vec_encoders import Seq2VecEncoder, PytorchSeq2VecWrapper
from allennlp.nn.util import get_text_field_mask
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder

class BaselineModel(Model):
    def __init__(self, word_embeddings: TextFieldEmbedder,
                 encoder: Seq2VecEncoder,
                 out_sz: int=len(label_cols)):
        super().__init__(vocab)
        self.word_embeddings = word_embeddings
        self.encoder = encoder
        self.projection = nn.Linear(self.encoder.get_output_dim(), out_sz)
        self.loss = nn.BCEWithLogitsLoss()
        
    def forward(self, tokens: Dict[str, torch.Tensor],
                id: Any, label: torch.Tensor) -> torch.Tensor:
        mask = get_text_field_mask(tokens)
        embeddings = self.word_embeddings(tokens)
        state = self.encoder(embeddings, mask)
        class_logits = self.projection(state)
        
        output = {"class_logits": class_logits}
        output["loss"] = self.loss(class_logits, label)

        return output

### Prepare embeddings

In [27]:
from allennlp.modules.text_field_embedders import BasicTextFieldEmbedder
from allennlp.modules.token_embedders import ElmoTokenEmbedder

options_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_options.json'
weight_file = 'https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x1024_128_2048cnn_1xhighway/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5'

elmo_embedder = ElmoTokenEmbedder(options_file, weight_file)
word_embeddings = BasicTextFieldEmbedder({"tokens": elmo_embedder})

100%|██████████| 54402456/54402456 [00:15<00:00, 3448186.09B/s]


In [28]:
from allennlp.modules.seq2vec_encoders import PytorchSeq2VecWrapper
encoder: Seq2VecEncoder = PytorchSeq2VecWrapper(nn.LSTM(word_embeddings.get_output_dim(), config.hidden_sz, bidirectional=True, batch_first=True))

Notice how simple and modular the code for initializing the model is. All the complexity is delegated to each component.

In [29]:
model = BaselineModel(
    word_embeddings, 
    encoder, 
)

In [30]:
if USE_GPU: model.cuda()
else: model

# Basic sanity checks

In [31]:
batch = nn_util.move_to_device(batch, 0 if USE_GPU else -1)

In [32]:
tokens = batch["tokens"]
labels = batch

In [33]:
tokens

{'tokens': tensor([[[259,  73, 112,  ..., 261, 261, 261],
          [259, 100,  98,  ..., 261, 261, 261],
          [259, 112, 111,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259, 118, 260,  ..., 261, 261, 261],
          [259, 115, 260,  ..., 261, 261, 261],
          [259,  98, 260,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         [[259,  74, 260,  ..., 261, 261, 261],
          [259,  71,  74,  ..., 261, 261, 261],
          [259,  74,  85,  ..., 261, 261, 261],
          ...,
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0],
          [  0,   0,   0,  ...,   0,   0,   0]],
 
         ...,
 
         [[259,  35, 260,  ..., 261, 261, 261],
        

In [34]:
mask = get_text_field_mask(tokens)
mask

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])

In [35]:
embeddings = model.word_embeddings(tokens)
state = model.encoder(embeddings, mask)
class_logits = model.projection(state)
class_logits

tensor([[-0.0529, -0.0262,  0.0334,  0.0932,  0.0485, -0.1344],
        [-0.2047,  0.1432,  0.2211, -0.0746, -0.0116, -0.1761],
        [ 0.1327,  0.1339,  0.0710, -0.0759,  0.0915, -0.1714],
        [-0.1824,  0.0264, -0.0531, -0.0794, -0.0975, -0.0400],
        [ 0.1211, -0.0398,  0.1096, -0.0567,  0.0396, -0.1569],
        [ 0.1192, -0.0919, -0.0552, -0.1347,  0.0565, -0.2150],
        [-0.0318, -0.0296, -0.0416, -0.0455,  0.1911, -0.0015],
        [ 0.0815,  0.1096,  0.1019, -0.0069, -0.0878, -0.1423],
        [-0.0638,  0.0486,  0.0728, -0.0058,  0.1464, -0.0970],
        [-0.0033,  0.0701,  0.1995, -0.0891, -0.0786, -0.1392],
        [ 0.0557,  0.0172,  0.1095,  0.0373,  0.1495, -0.0161],
        [ 0.0774, -0.1143,  0.0411, -0.0092,  0.1410, -0.1466],
        [-0.0185, -0.0398,  0.0511,  0.0097,  0.1832, -0.0019],
        [ 0.0347, -0.0351,  0.1361, -0.0501,  0.0505, -0.1964],
        [ 0.0262,  0.0115, -0.0532,  0.0221,  0.1430,  0.0151],
        [ 0.0248,  0.0891,  0.1217,  0.0

In [36]:
model(**batch)

{'class_logits': tensor([[-0.0789, -0.1261, -0.0641,  0.2692,  0.0014, -0.1165],
         [-0.0858, -0.0294,  0.2109, -0.1133,  0.1685, -0.1553],
         [ 0.0456, -0.0177, -0.0270, -0.0888,  0.1982, -0.3067],
         [-0.2441,  0.1275, -0.0710,  0.0294, -0.1422,  0.0463],
         [ 0.0972,  0.1346,  0.0665, -0.1366, -0.0044, -0.1929],
         [ 0.0320,  0.0629,  0.1336, -0.1517,  0.0499, -0.1969],
         [-0.1300,  0.0034, -0.0815, -0.0457,  0.0362, -0.1183],
         [ 0.0479,  0.0893,  0.1817, -0.0265,  0.0573, -0.1004],
         [ 0.0051, -0.0023,  0.0390,  0.0055,  0.0821,  0.0788],
         [ 0.0413,  0.1456,  0.2980,  0.1062, -0.1394, -0.1347],
         [-0.0863,  0.0092,  0.1713,  0.1499,  0.0142,  0.0188],
         [ 0.0976,  0.0484,  0.1318, -0.0328,  0.1388, -0.2082],
         [-0.0710, -0.0588,  0.1363, -0.0134,  0.1604, -0.2077],
         [-0.1084, -0.0315,  0.1354,  0.0544,  0.2159, -0.2088],
         [ 0.1554, -0.0027, -0.0675,  0.0137,  0.1493, -0.0789],
         

In [37]:
loss = model(**batch)["loss"]

In [38]:
loss

tensor(0.6976, grad_fn=<MeanBackward1>)

In [39]:
loss.backward()

In [40]:
[x.grad for x in list(model.encoder.parameters())]

[tensor([[-9.9208e-06,  9.9129e-05,  5.0682e-05,  ..., -8.7041e-05,
           3.5600e-05,  7.6566e-06],
         [-1.1369e-04, -5.1152e-05, -7.5502e-05,  ..., -1.1979e-04,
           1.2605e-04, -1.8761e-05],
         [-1.3619e-04,  1.0717e-04,  2.7693e-05,  ...,  5.1606e-05,
           2.1356e-04,  4.0541e-05],
         ...,
         [ 1.8521e-04,  1.4536e-05,  3.9201e-05,  ...,  1.8680e-05,
          -2.3767e-04,  1.4379e-05],
         [ 7.9033e-05,  1.5386e-05,  2.5557e-05,  ..., -6.1518e-05,
          -7.9275e-05, -1.4077e-05],
         [ 1.7141e-04, -4.5310e-07,  5.8982e-05,  ..., -1.1607e-04,
          -2.5490e-04,  8.1655e-05]]),
 tensor([[ 1.5749e-05,  8.9245e-06, -1.5019e-05,  ..., -5.9032e-06,
          -4.1844e-08, -1.0729e-05],
         [-2.6297e-05, -5.4704e-07, -4.8005e-05,  ..., -3.4190e-05,
          -4.1296e-05, -5.2007e-06],
         [ 5.1598e-05, -8.1258e-06, -8.2521e-05,  ..., -2.8385e-05,
           2.3811e-05,  4.0387e-05],
         ...,
         [ 3.0995e-05,  5

# Train

In [41]:
optimizer = optim.Adam(model.parameters(), lr=config.lr)

In [42]:
from allennlp.training.trainer import Trainer

trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_ds,
    cuda_device=0 if USE_GPU else -1,
    num_epochs=config.epochs,
)

In [43]:
metrics = trainer.train()

loss: 0.6856 ||: 100%|██████████| 5/5 [00:29<00:00,  6.91s/it]
loss: 0.6498 ||: 100%|██████████| 5/5 [00:29<00:00,  6.10s/it]


# Generating Predictions

In [44]:
from allennlp.data.iterators import DataIterator
from tqdm import tqdm
from scipy.special import expit # the sigmoid function

def tonp(tsr): return tsr.detach().cpu().numpy()

class Predictor:
    def __init__(self, model: Model, iterator: DataIterator,
                 cuda_device: int=-1) -> None:
        self.model = model
        self.iterator = iterator
        self.cuda_device = cuda_device
        
    def _extract_data(self, batch) -> np.ndarray:
        out_dict = self.model(**batch)
        return expit(tonp(out_dict["class_logits"]))
    
    def predict(self, ds: Iterable[Instance]) -> np.ndarray:
        pred_generator = self.iterator(ds, num_epochs=1, shuffle=False)
        self.model.eval()
        pred_generator_tqdm = tqdm(pred_generator,
                                   total=self.iterator.get_num_batches(ds))
        preds = []
        with torch.no_grad():
            for batch in pred_generator_tqdm:
                batch = nn_util.move_to_device(batch, self.cuda_device)
                preds.append(self._extract_data(batch))
        return np.concatenate(preds, axis=0)

In [45]:
from allennlp.data.iterators import BasicIterator
# iterate over the dataset without changing its order
seq_iterator = BasicIterator(batch_size=64)
seq_iterator.index_with(vocab)

In [46]:
predictor = Predictor(model, seq_iterator, cuda_device=0 if USE_GPU else -1)
train_preds = predictor.predict(train_ds) 
test_preds = predictor.predict(test_ds) 

100%|██████████| 4/4 [00:34<00:00,  8.58s/it]
